In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
pd.set_option('display.max_colwidth', 25)
import re
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

In [4]:
df = pd.read_csv("titanic_datasets/train.csv")
train_data = df.copy()
train_features = ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age','SibSp', 'Parch', 'Ticket', 'Fare',	'Cabin', 'Embarked']
test_feature = ['Survived']
# print(len(train_data))
# print(len(test_data))

In [5]:
test_data = pd.read_csv("titanic_datasets/test.csv")

In [6]:
def split_on_last_name(train_data_sub):
    last_name_split_sal = train_data_sub['Name'].str.split('[,.]', expand=True)
    last_name_split_sal[3] = last_name_split_sal[3].apply(lambda s :  '' if s is None else str(s))
    last_name_split_sal['First/Middle Name'] = [ "{} {}".format(x, y) for x,y in zip(last_name_split_sal[2], last_name_split_sal[3])]
    last_name_split_sal = last_name_split_sal.drop(columns=[2,3])
    last_name_split_sal.columns = ['Last Name', 'Salutation', 'First/Middle Name']
    last_name_split_sal = last_name_split_sal.applymap(lambda x: x.strip())
    return  train_data_sub.join(last_name_split_sal)

In [7]:
train_data = split_on_last_name(train_data)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Last Name,Salutation,First/Middle Name
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Braund,Mr,Owen Harris
1,2,1,1,"Cumings, Mrs. John Br...",female,38.0,1,0,PC 17599,71.2833,C85,C,Cumings,Mrs,John Bradley (Florenc...
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Heikkinen,Miss,Laina
3,4,1,1,"Futrelle, Mrs. Jacque...",female,35.0,1,0,113803,53.1000,C123,S,Futrelle,Mrs,Jacques Heath (Lily M...
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Allen,Mr,William Henry


In [8]:
train_data.set_index(['Last Name', 'Sex' , 'Salutation']).sort_index().head()

PassengerId  Survived  Pclass  \
Last Name Sex    Salutation                                  
Abbing    male   Mr                  846         0       3   
Abbott    female Mrs                 280         1       3   
          male   Mr                  747         0       3   
Abelson   female Mrs                 875         1       2   
          male   Mr                  309         0       2   

                                                 Name   Age  SibSp  Parch  \
Last Name Sex    Salutation                                                 
Abbing    male   Mr               Abbing, Mr. Anthony  42.0      0      0   
Abbott    female Mrs         Abbott, Mrs. Stanton ...  35.0      1      1   
          male   Mr          Abbott, Mr. Rossmore ...  16.0      1      1   
Abelson   female Mrs         Abelson, Mrs. Samuel ...  28.0      1      0   
          male   Mr               Abelson, Mr. Samuel  30.0      1      0   

                                Ticket   Fare Cabin Embarked  \
Last Name Sex    Salutation                                    
Abbing    male   Mr          C.A. 5547   7.55   NaN        S   
Abbott    female Mrs         C.A. 2673  20.25   NaN        S   
          male   Mr          C.A. 2673  20.25   NaN        S   
Abelson   female Mrs         P/PP 3381  24.00   NaN        C   
          male   Mr          P/PP 3381  24.00   NaN        C   

                                   First/Middle Name  
Last Name Sex    Salutation                           
Abbing    male   Mr                          Anthony  
Abbott    female Mrs             Stanton (Rosa Hunt)  
          male   Mr                  Rossmore Edward  
Abelson   female Mrs         Samuel (Hannah Wizosky)  
          male   Mr                           Samuel

In [9]:
features = ['PassengerId', 'Last Name', 'Salutation', 'First/Middle Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Pclass', 'Fare', 'Ticket', 'Cabin', 'Embarked' ]
train_labels = train_data[['Survived']]
train_features = train_data[features]
test_features = test_data[['PassengerId',  'Sex', 'Age', 'SibSp', 'Parch', 'Pclass', 'Embarked', 'Ticket']]

In [10]:
# Not used at moment:  'Fare', 'Ticket', 'Last Name', 'Salutation', 'First/Middle Name','Cabin',

In [11]:
def classify(train_features_reduced):
    X_train, X_test, y_train, y_test = train_test_split(train_features_reduced.drop(columns=['PassengerId']), train_labels['Survived'], test_size=0.2)
    model = RandomForestClassifier(
                n_estimators=150,
                max_depth=20,
                random_state=1)
    model.fit(X_train, y_train)
    return (model , model.score(X_test, y_test))

def output(train_features_reduced, test_X):
    res = classify(train_features_reduced)
    model = res[0]
    predictions = model.predict(test_X)
    predictions.to_csv('Submission3.csv', index=False)

### Cross Validation Score

In [12]:
def cross_validate(train_features_reduced):
    res = classify(train_features_reduced)
    model = res[0]
    scores = cross_val_score(model, train_features_reduced.drop(columns=['PassengerId']), train_labels['Survived'], cv=20)
    return  'Model Score: {mscore}, Cross Val Score: {cvscore}'.format(mscore=res[1], cvscore=np.mean(scores))

In [13]:
train_features['Age'] = train_features['Age'].fillna(np.mean(train_data['Age']), inplace = True)

C:\Users\divjo\Anaconda3\envs\super\lib\site-packages\pandas\core\generic.py:6244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\divjo\Anaconda3\envs\super\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
train_features_reduced = pd.get_dummies(train_features[['PassengerId',  'Sex', 'Age', 'SibSp', 'Parch', 'Pclass',  'Embarked' ]])
cross_validate(train_features_reduced)

'Model Score: 0.7877094972067039, Cross Val Score: 0.7967929292929293'

#### Let us improve accuracy by including other features

In [52]:
train_features_reduced = pd.get_dummies(train_features[['PassengerId',  'Sex', 'Age', 'SibSp', 'Parch', 'Pclass', 'Embarked', 'Ticket']])
cross_validate(train_features_reduced)

'Model Score: 0.7877094972067039, Cross Val Score: 0.8135858585858585'

#### Using this, lets output

In [53]:
test_features['Age'] = test_features['Age'].fillna(np.mean(test_data['Age']), inplace = True)
test_features_reduced = pd.get_dummies(test_features[['PassengerId',  'Sex', 'Age', 'SibSp', 'Parch', 'Pclass', 'Embarked', 'Ticket']])

C:\Users\divjo\Miniconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\Users\divjo\Miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


#### !! Important !! Ticket is such a columns - the values of which are not categorical nor all the value type are there in train data and some of the ticket values in test are not in train data

In [80]:
print(len(list(set(test_data['Ticket']) - set(train_data['Ticket']))))## TRICKKY!!!!
print(len(list(set(train_data ['Ticket']) - set(test_data['Ticket']))))## TRICKKY!!!!
# So that means we cannot one hot encode Tickets as categorical  :(

248
566


train_data[]